# In this notebook we would create the LIGHTGBM model


In [ ]:
DROP_COLS = [ 
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'volume_breakout', 'volume_breakdown', 'break_upper_band', 'break_lower_band',
    'vol_spike_1_5x', 'rsi_oversold', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'near_upper_band',
    'near_lower_band', 'overbought_reversal', 'oversold_reversal',
    'ema_cross_up', 'ema_cross_down', 'macd_cross_up', 'macd_cross_down',
    'trending_market', 'trend_alignment', 'ema7_above_ema21', 'macd_rising',
    'bollinger_upper', 'bollinger_lower', 'bullish_scenario_1',
    'bullish_scenario_5', 'bearish_scenario_1'
]

In [18]:
import numpy as np
import pandas as pd
import optuna
import json
import warnings
from pathlib import Path
from sklearn.metrics import fbeta_score
from sklearn.model_selection import TimeSeriesSplit
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping

# ───── CONFIG ─────────────────────────────────────────
SEED         = 42
FOLDS        = 3
GAP_ROWS     = 12
EARLY_STOP   = 200
N_TRIALS     = 60
TIMEOUT      = 1800  # seconds (30 min max)
MAX_TREES    = 2500
BETA         = 2
CSV_PATH     = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

DROP_COLS = [
    'open', 'high', 'low', 'high_low', 'high_close', 'low_close', 'typical_price',
    'volume_mean_20', 'EMA_21', 'SMA_20', 'vwap_24h', 'bollinger_upper', 'bollinger_lower',
    'MACD_line', 'MACD_signal', 'resistance_level', 'support_level', 'vol_spike_1_5x',
    'near_upper_band', 'near_lower_band', 'break_upper_band', 'break_lower_band',
    'rsi_oversold', 'above_sma20', 'macd_positive', 'volume_breakout', 'volume_breakdown',
    'above_sma50', 'ema7_above_ema21', 'ema_cross_down', 'rsi_overbought', 'stoch_overbought',
    'stoch_oversold', 'cci_overbought', 'cci_oversold', 'trending_market',
    'bullish_scenario_1', 'bullish_scenario_2', 'bullish_scenario_3', 'bullish_scenario_4',
    'bullish_scenario_5', 'bullish_scenario_6', 'bearish_scenario_1', 'bearish_scenario_2',
    'bearish_scenario_3', 'bearish_scenario_4', 'bearish_scenario_6', 'close'
]

warnings.filterwarnings("ignore")
np.random.seed(SEED)

# ───── Load & preprocess data ───────────────────────
def load_data():
    df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
    df = df[df.index >= "2018-01-01"]
    df.drop(columns=[c for c in DROP_COLS if c in df.columns], inplace=True)
    df = df[df["target"].notna()]
    df = df.dropna().select_dtypes(include=[np.number])
    X = df.drop(columns=["target"]).values
    y = df["target"].astype(int).values
    return X, y

# ───── Custom F2 eval function ───────────────────────
def f2_score_eval(y_true, y_pred):
    y_pred_binary = (y_pred >= 0.5).astype(int)
    f2 = fbeta_score(y_true, y_pred_binary, beta=BETA, zero_division=0)
    return 'f2', f2, True

# ───── Objective function ─────────────────────────────
def objective(trial, data):
    X, y = data
    boosting_type = trial.suggest_categorical("boosting_type", ["gbdt", "dart"])

    params = {
        "boosting_type": boosting_type,
        "device":        "gpu",
        "gpu_use_dp":    False,
        "random_state":  SEED,
        "verbosity":     -1,

        # Optimized search space
        "learning_rate":     trial.suggest_float("lr", 0.02, 0.2, log=True),
        "num_leaves":        trial.suggest_int("leaves", 31, 256, step=8),
        "max_depth":         trial.suggest_int("depth", 4, 12),
        "min_child_samples": trial.suggest_int("min_child", 20, 300),
        "feature_fraction":  trial.suggest_float("feat_frac", 0.6, 1.0),
        "bagging_fraction":  trial.suggest_float("bag_frac", 0.6, 1.0),
        "bagging_freq":      trial.suggest_int("bag_freq", 1, 5),
        "reg_alpha":         trial.suggest_float("l1", 0.0, 6.0),
        "reg_lambda":        trial.suggest_float("l2", 0.0, 6.0),
        "min_split_gain":    trial.suggest_float("gamma", 0.0, 3.0),
        "extra_trees":       trial.suggest_categorical("extra_t", [True, False]),
        "max_bin":           trial.suggest_int("max_bin", 127, 255)
    }

    if boosting_type == "dart":
        params.update({
            "drop_rate": trial.suggest_float("drop", 0.05, 0.4),
            "skip_drop": trial.suggest_float("skip", 0.2, 0.8)
        })

    cv = TimeSeriesSplit(n_splits=FOLDS, gap=GAP_ROWS)
    scores = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        clf = LGBMClassifier(**params, n_estimators=MAX_TREES)

        clf.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            eval_metric=f2_score_eval,
            callbacks=[early_stopping(EARLY_STOP, verbose=False)]
        )

        preds = clf.predict(X_val)
        f2 = fbeta_score(y_val, preds, beta=BETA, zero_division=0)
        scores.append(f2)

        trial.report(f2, fold)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return -np.mean(scores)

# ───── Optuna runner ─────────────────────────────
def run_optuna():
    data = load_data()
    study = optuna.create_study(
        direction="minimize",
        sampler=optuna.samplers.TPESampler(seed=SEED),
        pruner=optuna.pruners.SuccessiveHalvingPruner(
            min_resource=1,
            reduction_factor=3,
            min_early_stopping_rate=0
        ),
        study_name="lgbm_gpu_fast"
    )

    print("\n🚀 Starting LightGBM GPU Optimization...")
    study.optimize(lambda trial: objective(trial, data), n_trials=N_TRIALS, timeout=TIMEOUT, show_progress_bar=True)

    print("\n🏆 Best F2 score:", -study.best_value)
    print("🔧 Best params:")
    for k, v in study.best_params.items():
        print(f"  {k:20}: {v}")

    with open("best_lgbm_params.json", "w") as f:
        json.dump(study.best_params, f, indent=2)

if __name__ == "__main__":
    run_optuna()


[I 2025-06-06 19:38:17,572] A new study created in memory with name: lgbm_gpu_fast



🚀 Starting LightGBM GPU Optimization...


Best trial: 0. Best value: -0.444346:   2%|▏         | 1/60 [00:26<25:55, 26.37s/it, 26.37/1800 seconds]

[I 2025-06-06 19:38:43,937] Trial 0 finished with value: -0.4443464670992617 and parameters: {'boosting_type': 'dart', 'lr': 0.10790061933340454, 'leaves': 167, 'depth': 5, 'min_child': 63, 'feat_frac': 0.6232334448672797, 'bag_frac': 0.9464704583099741, 'bag_freq': 4, 'l1': 4.248435466776273, 'l2': 0.12350696577481468, 'gamma': 2.909729556485983, 'extra_t': True, 'max_bin': 150, 'drop': 0.11419157844870184, 'skip': 0.38254534577572263}. Best is trial 0 with value: -0.4443464670992617.


Best trial: 0. Best value: -0.444346:   3%|▎         | 2/60 [00:28<11:41, 12.10s/it, 28.48/1800 seconds]

...

[I 2025-06-06 19:59:43,410] Trial 57 finished with value: -0.5218909908319844 and parameters: {'boosting_type': 'dart', 'lr': 0.14823970926612529, 'leaves': 247, 'depth': 10, 'min_child': 225, 'feat_frac': 0.9803010207329786, 'bag_frac': 0.6647017582754398, 'bag_freq': 1, 'l1': 1.838193186625527, 'l2': 2.3507208400940076, 'gamma': 1.4409611118918684, 'extra_t': True, 'max_bin': 184, 'drop': 0.251556402496246, 'skip': 0.2412691572429682}. Best is trial 53 with value: -0.580690514282784.


Best trial: 53. Best value: -0.580691:  98%|█████████▊| 59/60 [21:38<00:19, 19.09s/it, 1298.80/1800 seconds]

[I 2025-06-06 19:59:56,372] Trial 58 pruned. 


Best trial: 53. Best value: -0.580691: 100%|██████████| 60/60 [21:55<00:00, 21.92s/it, 1315.38/1800 seconds]

[I 2025-06-06 20:00:12,951] Trial 59 pruned. 

🏆 Best F2 score: 0.580690514282784
🔧 Best params:
  boosting_type       : dart
  lr                  : 0.19857041309331078
  leaves              : 255
  depth               : 9
  min_child           : 172
  feat_frac           : 0.9754014272237063
  bag_frac            : 0.66931809977155
  bag_freq            : 1
  l1                  : 1.9563928569633862
  l2                  : 0.7561089183959027
  gamma               : 2.1785543202146207
  extra_t             : True
  max_bin             : 175
  drop                : 0.22119536646782006
  skip                : 0.2483700005269152


In [19]:
"""
lgbm_dart_final.py
==================
Train an optimal LightGBM-DART model for 4-hour BTC direction.

Requires:
    pip install lightgbm>=3.3 numpy pandas scikit-learn
    # GPU: build LightGBM with CUDA support
"""

import numpy as np
import pandas as pd
import json, time, warnings
from pathlib import Path
from sklearn.metrics import (
    fbeta_score, accuracy_score, precision_recall_fscore_support, roc_auc_score
)
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation

warnings.filterwarnings("ignore")

# ───── Config ──────────────────────────────────────────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

DROP_COLS = [
    'open','high','low','close','high_low','high_close','low_close','true_range',
    'typical_price','volume_mean_20','EMA_21','SMA_20','vwap_24h','close_4h',
    'bollinger_upper','bollinger_lower','MACD_line','MACD_signal',
    'resistance_level','support_level','vol_spike_1_5x','near_upper_band',
    'near_lower_band','break_upper_band','break_lower_band','rsi_oversold',
    'rsi_overbought','above_sma20','macd_positive','volume_breakout',
    'volume_breakdown','above_sma50','ema7_above_ema21','ema_cross_down',
    'ema_cross_up','stoch_overbought','stoch_oversold','cci_overbought',
    'cci_oversold','trending_market','trend_alignment','macd_cross_up',
    'macd_cross_down','macd_rising','overbought_reversal','oversold_reversal',
    'bullish_scenario_1','bullish_scenario_2','bullish_scenario_3',
    'bullish_scenario_4','bullish_scenario_5','bullish_scenario_6',
    'bearish_scenario_1','bearish_scenario_2','bearish_scenario_3',
    'bearish_scenario_4','bearish_scenario_6','timestamp','date','Unnamed: 0'
]

TEST_FRACTION      = 0.20   # chronological split
EARLY_STOP_ROUNDS  = 300
N_ESTIMATORS       = 3000
BETA               = 2.0
SEED               = 42

OPTIMAL_PARAMS = dict(
    boosting_type   = "dart",
    learning_rate   = 0.19857041309331078,
    num_leaves      = 255,
    max_depth       = 9,
    min_child_samples = 172,
    feature_fraction= 0.9754014272237063,
    bagging_fraction= 0.66931809977155,
    bagging_freq    = 1,
    reg_alpha       = 1.9563928569633862,
    reg_lambda      = 0.7561089183959027,
    min_split_gain  = 2.1785543202146207,
    extra_trees     = True,
    max_bin         = 175,
    drop_rate       = 0.22119536646782006,
    skip_drop       = 0.2483700005269152,
    objective       = "binary",
    n_estimators    = N_ESTIMATORS,
    random_state    = SEED,
    verbosity       = -1,
    device          = "gpu"     # auto-fallback to CPU handled below
)

# ───── Data load ───────────────────────────────────────────────────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2020-01-01"]
df.drop(columns=[c for c in DROP_COLS if c in df.columns], inplace=True)
df = df[df["target"].notna()].dropna()

X = df.drop(columns="target")
y = df["target"].astype(int).values
dates = df.index

split = int(len(df) * (1 - TEST_FRACTION))
X_train, X_val = X.iloc[:split], X.iloc[split:]
y_train, y_val = y[:split],      y[split:]
dates_val      = dates[split:]

print(f"Train rows: {len(X_train)} | Val rows: {len(X_val)}")

# ───── Model fit ───────────────────────────────────────────────────────
def fit_lightgbm(params):
    model = LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="binary_logloss",
        callbacks=[
            early_stopping(EARLY_STOP_ROUNDS, verbose=False),
            log_evaluation(period=200)
        ]
    )
    return model

try:
    print("Trying GPU training …")
    model = fit_lightgbm(OPTIMAL_PARAMS)
except Exception as gpu_err:
    print("⚠ GPU failed:", gpu_err, "\n→ Falling back to CPU.")
    cpu_params = OPTIMAL_PARAMS.copy(); cpu_params["device"] = "cpu"
    model = fit_lightgbm(cpu_params)

print(f"Best iteration: {model.best_iteration_}")

# ───── Evaluation ─────────────────────────────────────────────────────
prob = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
pred = (prob >= 0.5).astype(int)

acc  = accuracy_score(y_val, pred)
f2   = fbeta_score(y_val, pred, beta=BETA, zero_division=0)
auc  = roc_auc_score(y_val, prob)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_val, pred, labels=[0,1], zero_division=0
)

print("\n── Validation ──")
print(f"Accuracy : {acc:.3f}")
print(f"AUC      : {auc:.3f}")
print(f"F{int(BETA)}-score : {f2:.3f}")

# ───── Feature importance ─────────────────────────────────────────────
imp_df = pd.DataFrame({
    "feature": X.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

print("\nTop 10 features")
print(imp_df.head(10).to_string(index=False))

# ───── Save artefacts ─────────────────────────────────────────────────
model.booster_.save_model("lgbm_btc_direction_optimal.txt")
pd.DataFrame({
    "prob_up": prob,
    "pred_0.50": pred,
    "actual": y_val
}, index=dates_val).to_csv("lgbm_optimal_predictions.csv")
imp_df.to_csv("lgbm_feature_importance.csv", index=False)

summary = {
    "metrics": {
        "accuracy": float(acc),
        "auc":      float(auc),
        "f2":       float(f2),
        "precision_0": float(prec[0]),
        "precision_1": float(prec[1]),
        "recall_0":    float(rec[0]),
        "recall_1":    float(rec[1])
    },
    "best_iteration": int(model.best_iteration_),
    "n_features": int(X.shape[1]),
    "train_rows": int(len(X_train)),
    "val_rows": int(len(X_val)),
    "top_features": imp_df.head(15)["feature"].tolist(),
    "params": OPTIMAL_PARAMS
}

with open("lgbm_training_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n✔ All artefacts saved")


Train rows: 9180 | Val rows: 2296
Trying GPU training …
[200]	valid_0's binary_logloss: 0.690121
[400]	valid_0's binary_logloss: 0.691164
[600]	valid_0's binary_logloss: 0.695676
[800]	valid_0's binary_logloss: 0.699606
[1000]	valid_0's binary_logloss: 0.701675
[1200]	valid_0's binary_logloss: 0.703902
[1400]	valid_0's binary_logloss: 0.705691
[1600]	valid_0's binary_logloss: 0.707865
[1800]	valid_0's binary_logloss: 0.713544
[2000]	valid_0's binary_logloss: 0.715526
[2200]	valid_0's binary_logloss: 0.717318
[2400]	valid_0's binary_logloss: 0.714333
[2600]	valid_0's binary_logloss: 0.716142
[2800]	valid_0's binary_logloss: 0.716964
[3000]	valid_0's binary_logloss: 0.714762
Best iteration: 0

── Validation ──
Accuracy : 0.494
AUC      : 0.504
F2-score : 0.136

Top 10 features
         feature  importance
          roc_4h         324
 buying_pressure         223
             CCI         165
             OBV         143
  obv_rising_24h         138
fear_greed_score         137
         ro

In [21]:
"""
lgbm_dart_final.py
==================
Final LightGBM-DART model for 4h BTC direction, trained on 2018+ data.
Validation is strictly the last 20% (starting 2023-08-08), to mimic real deployment.
"""

import numpy as np
import pandas as pd
import warnings, json
from pathlib import Path
from sklearn.metrics import (
    fbeta_score, accuracy_score, precision_recall_fscore_support, roc_auc_score
)
from lightgbm import LGBMClassifier
from lightgbm.callback import early_stopping, log_evaluation

warnings.filterwarnings("ignore")

# ───── Configuration ─────────────────────────────────────────────────────
CSV_PATH = Path(r"C:\Users\ADMIN\Desktop\Coding_projects\stock_market_prediction"
                r"\Stock-Market-Prediction\data\processed\gemini_btc_with_features_4h.csv")

DROP_COLS = [
    'open','high','low','close','high_low','high_close','low_close','true_range',
    'typical_price','volume_mean_20','EMA_21','SMA_20','vwap_24h','close_4h',
    'bollinger_upper','bollinger_lower','MACD_line','MACD_signal',
    'resistance_level','support_level','vol_spike_1_5x','near_upper_band',
    'near_lower_band','break_upper_band','break_lower_band','rsi_oversold',
    'rsi_overbought','above_sma20','macd_positive','volume_breakout',
    'volume_breakdown','above_sma50','ema7_above_ema21','ema_cross_down',
    'ema_cross_up','stoch_overbought','stoch_oversold','cci_overbought',
    'cci_oversold','trending_market','trend_alignment','macd_cross_up',
    'macd_cross_down','macd_rising','overbought_reversal','oversold_reversal',
    'bullish_scenario_1','bullish_scenario_2','bullish_scenario_3',
    'bullish_scenario_4','bullish_scenario_5','bullish_scenario_6',
    'bearish_scenario_1','bearish_scenario_2','bearish_scenario_3',
    'bearish_scenario_4','bearish_scenario_6','timestamp','date','Unnamed: 0'
]

VALIDATION_START_DATE = "2023-08-08"
BETA               = 2.0
EARLY_STOP_ROUNDS  = 300
N_ESTIMATORS       = 3000
SEED               = 42

# Optimal params (lowered LR slightly for stability)
PARAMS = dict(
    boosting_type   = "dart",
    learning_rate   = 0.09,  # lowered from 0.19 for better generalization
    num_leaves      = 255,
    max_depth       = 9,
    min_child_samples = 172,
    feature_fraction= 0.975,
    bagging_fraction= 0.669,
    bagging_freq    = 1,
    reg_alpha       = 1.956,
    reg_lambda      = 0.756,
    min_split_gain  = 2.178,
    extra_trees     = True,
    max_bin         = 175,
    drop_rate       = 0.221,
    skip_drop       = 0.248,
    objective       = "binary",
    n_estimators    = N_ESTIMATORS,
    random_state    = SEED,
    verbosity       = -1,
    device          = "gpu"
)

# ───── Data Loading ─────────────────────────────────────────────────────
df = pd.read_csv(CSV_PATH, index_col=0, parse_dates=True)
df = df[df.index >= "2018-01-01"]
df.drop(columns=[c for c in DROP_COLS if c in df.columns], inplace=True)
df = df[df["target"].notna()].dropna()

X = df.drop(columns="target")
y = df["target"].astype(int).values
dates = df.index

# ───── Train/Validation Split ───────────────────────────────────────────
val_start = pd.to_datetime(VALIDATION_START_DATE)
X_train, X_val = X[dates < val_start], X[dates >= val_start]
y_train, y_val = y[dates < val_start], y[dates >= val_start]
dates_val = dates[dates >= val_start]

print(f"Train rows: {len(X_train)} | Val rows: {len(X_val)}")

# ───── Training ─────────────────────────────────────────────────────────
def train_model(params):
    model = LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric="binary_logloss",
        callbacks=[
            early_stopping(EARLY_STOP_ROUNDS, verbose=False),
            log_evaluation(period=200)
        ]
    )
    return model

try:
    print("🚀 Training with GPU …")
    model = train_model(PARAMS)
except Exception as err:
    print("⚠ GPU failed:", err, "\n→ Falling back to CPU.")
    cpu_params = PARAMS.copy(); cpu_params["device"] = "cpu"
    model = train_model(cpu_params)

print(f"Best iteration: {model.best_iteration_}")

# ───── Evaluation ──────────────────────────────────────────────────────
prob = model.predict_proba(X_val, num_iteration=model.best_iteration_)[:, 1]
pred = (prob >= 0.5).astype(int)

acc  = accuracy_score(y_val, pred)
f2   = fbeta_score(y_val, pred, beta=BETA, zero_division=0)
auc  = roc_auc_score(y_val, prob)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_val, pred, labels=[0,1], zero_division=0
)

print("\n── Validation ──")
print(f"Accuracy : {acc:.3f}")
print(f"AUC      : {auc:.3f}")
print(f"F{int(BETA)}-score : {f2:.3f}")
print(f"Precision (1): {prec[1]:.3f} | Recall (1): {rec[1]:.3f} | F1 (1): {f1[1]:.3f}")

# ───── Save Results ────────────────────────────────────────────────────
pd.DataFrame({
    "prob_up": prob,
    "pred_0.50": pred,
    "actual": y_val
}, index=dates_val).to_csv("lgbm_optimal_predictions.csv")

model.booster_.save_model("lgbm_btc_direction_optimal.txt")

imp_df = pd.DataFrame({
    "feature": X.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)
imp_df.to_csv("lgbm_feature_importance.csv", index=False)

summary = {
    "metrics": {
        "accuracy": float(acc),
        "auc":      float(auc),
        "f2":       float(f2),
        "precision_1": float(prec[1]),
        "recall_1":    float(rec[1])
    },
    "best_iteration": int(model.best_iteration_),
    "n_features": int(X.shape[1]),
    "train_rows": int(len(X_train)),
    "val_rows": int(len(X_val)),
    "top_features": imp_df.head(15)["feature"].tolist(),
    "params": PARAMS
}

with open("lgbm_training_summary.json", "w") as f:
    json.dump(summary, f, indent=2)

print("\n✔ All artefacts saved")


Train rows: 12266 | Val rows: 3589
🚀 Training with GPU …
[200]	valid_0's binary_logloss: 0.688358
[400]	valid_0's binary_logloss: 0.688617
[600]	valid_0's binary_logloss: 0.688596
[800]	valid_0's binary_logloss: 0.689414
[1000]	valid_0's binary_logloss: 0.69066
[1200]	valid_0's binary_logloss: 0.691415
[1400]	valid_0's binary_logloss: 0.691624
[1600]	valid_0's binary_logloss: 0.692386
[1800]	valid_0's binary_logloss: 0.69353
[2000]	valid_0's binary_logloss: 0.693635
[2200]	valid_0's binary_logloss: 0.694973
[2400]	valid_0's binary_logloss: 0.694637
[2600]	valid_0's binary_logloss: 0.695546
[2800]	valid_0's binary_logloss: 0.695825
[3000]	valid_0's binary_logloss: 0.696568
Best iteration: 0

── Validation ──
Accuracy : 0.527
AUC      : 0.537
F2-score : 0.422
Precision (1): 0.559 | Recall (1): 0.398 | F1 (1): 0.465

✔ All artefacts saved
